<div style="background: linear-gradient(135deg, #3b82f6, #06b6d4); color: white; padding: 20px; border-radius: 12px; text-align: center; font-family: 'Segoe UI', sans-serif;">

  <h2 style="margin-bottom: 10px;">🔬 Detecting Image Manipulation in Scientific Figures</h2>
  <p style="font-size: 15px; margin-top: 0;">
    A simple and clear notebook for identifying <b>copy-move forgeries</b> in biomedical images using semantic segmentation.
  </p>
</div>

<div style="background-color: #f8fafc; border-radius: 12px; padding: 18px; margin-top: 15px; font-family: 'Segoe UI', sans-serif;">
  <h3 style="color: #2563eb;">🎯 What We Do</h3>
  <p style="margin-top: -8px; color: #334155;">
    Build a segmentation model capable of locating duplicated regions within scientific figures.
  </p>

  <h3 style="color: #2563eb;">⚙️ Core Challenge</h3>
  <p style="margin-top: -8px; color: #334155;">
    The images vary in size and forged areas are small, demanding careful preprocessing and strong augmentations.
  </p>

  <h3 style="color: #2563eb;">🚀 Notebook Goal</h3>
  <p style="margin-top: -8px; color: #334155;">
    Provide a clean, beginner-friendly baseline for detecting manipulations with pixel-level precision.
  </p>
</div>

<div style="background: #e0f2fe; border-radius: 8px; padding: 10px; margin-top: 10px; text-align: center; font-size: 14px; color: #0369a1;">
  Ready to explore ➜ <b>EDA → Data Prep → Dta Augmentation → Modeling → Evaluation</b>
</div>


# Import necessary libraries

In [ ]:
import sys
sys.path.append("/kaggle/input/smp-lib-zip/") 

import warnings
warnings.filterwarnings("ignore")

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
import cv2
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import albumentations as A
from albumentations.pytorch import ToTensorV2
import segmentation_models_pytorch as smp
import warnings
import time
from collections import defaultdict
import copy
warnings.filterwarnings('ignore')
np.random.seed(42)
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")


# Data Loading and EDA

**This step loads the dataset and performs exploratory analysis to understand image dimensions, distributions, and mask properties. We visualize sample authentic and forged images, overlay forged regions, and examine key statistics like number of regions and forged area percentage, providing insights into the dataset and the challenge of detecting copy-move forgeries.**

In [ ]:
BASE_PATH = "/kaggle/input/recodai-luc-scientific-image-forgery-detection"
TRAIN_AUTHENTIC_PATH = os.path.join(BASE_PATH, "train_images/authentic")
TRAIN_FORGED_PATH = os.path.join(BASE_PATH, "train_images/forged")
TRAIN_MASKS_PATH = os.path.join(BASE_PATH, "train_masks")
TEST_IMAGES_PATH = os.path.join(BASE_PATH, "test_images")


print("="*60)
print(" DATA LOADING AND BASIC STATISTICS")
print("="*60)

authentic_images = sorted([f for f in os.listdir(TRAIN_AUTHENTIC_PATH) if f.endswith('.png')])
forged_images = sorted([f for f in os.listdir(TRAIN_FORGED_PATH) if f.endswith('.png')])
mask_files = sorted([f for f in os.listdir(TRAIN_MASKS_PATH) if f.endswith('.npy')])
test_images = sorted([f for f in os.listdir(TEST_IMAGES_PATH) if f.endswith('.png')])

print(f"Number of AUTHENTIC images: {len(authentic_images)}")
print(f"Number of FORGED images: {len(forged_images)}")
print(f"Number of MASK files: {len(mask_files)}")
print(f"Number of TEST images: {len(test_images)}")
print(f"\nTotal TRAIN images: {len(authentic_images) + len(forged_images)}")
print(f"Forged/Authentic ratio: {len(forged_images)/len(authentic_images):.2f}")
print("\n")

<h2 style="text-align:center; color:#2c3e50;">
   Analyzing Image Dimensions
</h2>
<hr style="width:50%; margin:auto; height:2px; background-color:#2c3e50; border:none;">


In [ ]:
def get_image_dimensions(image_path):
    img = Image.open(image_path)
    return img.size 
sample_size = min(100, len(authentic_images) + len(forged_images))

authentic_dims = []
forged_dims = []

print("Analyzing authentic images dimensions...")
for img_name in tqdm(authentic_images[:50], desc="Authentic"):
    img_path = os.path.join(TRAIN_AUTHENTIC_PATH, img_name)
    w, h = get_image_dimensions(img_path)
    authentic_dims.append((w, h, w*h))

print("Analyzing forged images dimensions...")
for img_name in tqdm(forged_images[:50], desc="Forged"):
    img_path = os.path.join(TRAIN_FORGED_PATH, img_name)
    w, h = get_image_dimensions(img_path)
    forged_dims.append((w, h, w*h))

authentic_dims = np.array(authentic_dims)
forged_dims = np.array(forged_dims)

print(f"\n--- AUTHENTIC IMAGES ---")
print(f"Width  - Min: {authentic_dims[:,0].min():.0f}, Max: {authentic_dims[:,0].max():.0f}, Mean: {authentic_dims[:,0].mean():.0f}")
print(f"Height - Min: {authentic_dims[:,1].min():.0f}, Max: {authentic_dims[:,1].max():.0f}, Mean: {authentic_dims[:,1].mean():.0f}")
print(f"Area   - Min: {authentic_dims[:,2].min():.0f}, Max: {authentic_dims[:,2].max():.0f}, Mean: {authentic_dims[:,2].mean():.0f}")

print(f"\n--- FORGED IMAGES ---")
print(f"Width  - Min: {forged_dims[:,0].min():.0f}, Max: {forged_dims[:,0].max():.0f}, Mean: {forged_dims[:,0].mean():.0f}")
print(f"Height - Min: {forged_dims[:,1].min():.0f}, Max: {forged_dims[:,1].max():.0f}, Mean: {forged_dims[:,1].mean():.0f}")
print(f"Area   - Min: {forged_dims[:,2].min():.0f}, Max: {forged_dims[:,2].max():.0f}, Mean: {forged_dims[:,2].mean():.0f}")
print("\n")
fig, axes = plt.subplots(2, 3, figsize=(18, 10))
fig.suptitle('Image Dimensions Distribution', fontsize=16, fontweight='bold')

axes[0, 0].hist(authentic_dims[:,0], bins=30, color='skyblue', edgecolor='black', alpha=0.7)
axes[0, 0].set_title('Authentic - Width Distribution')
axes[0, 0].set_xlabel('Width (pixels)')
axes[0, 0].set_ylabel('Frequency')

axes[0, 1].hist(authentic_dims[:,1], bins=30, color='lightcoral', edgecolor='black', alpha=0.7)
axes[0, 1].set_title('Authentic - Height Distribution')
axes[0, 1].set_xlabel('Height (pixels)')
axes[0, 1].set_ylabel('Frequency')

axes[0, 2].hist(authentic_dims[:,2], bins=30, color='lightgreen', edgecolor='black', alpha=0.7)
axes[0, 2].set_title('Authentic - Area Distribution')
axes[0, 2].set_xlabel('Area (pixels²)')
axes[0, 2].set_ylabel('Frequency')


axes[1, 0].hist(forged_dims[:,0], bins=30, color='skyblue', edgecolor='black', alpha=0.7)
axes[1, 0].set_title('Forged - Width Distribution')
axes[1, 0].set_xlabel('Width (pixels)')
axes[1, 0].set_ylabel('Frequency')

axes[1, 1].hist(forged_dims[:,1], bins=30, color='lightcoral', edgecolor='black', alpha=0.7)
axes[1, 1].set_title('Forged - Height Distribution')
axes[1, 1].set_xlabel('Height (pixels)')
axes[1, 1].set_ylabel('Frequency')

axes[1, 2].hist(forged_dims[:,2], bins=30, color='lightgreen', edgecolor='black', alpha=0.7)
axes[1, 2].set_title('Forged - Area Distribution')
axes[1, 2].set_xlabel('Area (pixels²)')
axes[1, 2].set_ylabel('Frequency')

plt.tight_layout()
plt.show()



<h2 style="text-align:center; color:#2c3e50;">
   VISUALIZING FORGED REGIONS WITH OVERLAY
</h2>
<hr style="width:50%; margin:auto; height:2px; background-color:#2c3e50; border:none;">


In [ ]:
def visualize_forgery_overlay(forged_list, masks_path, n_samples=4): 
    fig, axes = plt.subplots(n_samples, 3, figsize=(18, 6*n_samples))
    fig.suptitle('Forged Images with Mask Overlay', fontsize=16, fontweight='bold')
    
    for i in range(n_samples):
        img_path = os.path.join(TRAIN_FORGED_PATH, forged_list[i])
        img = np.array(Image.open(img_path))
        mask_name = forged_list[i].replace('.png', '.npy')
        mask_path = os.path.join(masks_path, mask_name)
        
        if os.path.exists(mask_path):
            mask = np.load(mask_path)
            if len(mask.shape) == 3:
                combined_mask = np.max(mask, axis=0)
                n_regions = mask.shape[0]
            else:
                combined_mask = mask
                n_regions = 1
            
            axes[i, 0].imshow(img)
            axes[i, 0].set_title(f'Original Image\n{forged_list[i]}', fontsize=10)
            axes[i, 0].axis('off')

            axes[i, 1].imshow(combined_mask, cmap='hot')
            axes[i, 1].set_title(f'Forgery Mask\n{n_regions} region(s)', fontsize=10)
            axes[i, 1].axis('off')
            
    
            overlay = img.copy()
            if len(img.shape) == 2:  
                overlay = cv2.cvtColor(overlay, cv2.COLOR_GRAY2RGB)
            
            red_mask = np.zeros_like(overlay)
            red_mask[:,:,0] = combined_mask * 255  
            alpha = 0.4
            blended = cv2.addWeighted(overlay, 1-alpha, red_mask, alpha, 0)
            
            axes[i, 2].imshow(blended)
            axes[i, 2].set_title('Overlay (Red = Forged)', fontsize=10)
            axes[i, 2].axis('off')
        else:
            for j in range(3):
                axes[i, j].text(0.5, 0.5, 'MASK NOT FOUND', 
                               ha='center', va='center', fontsize=12)
                axes[i, j].axis('off')
    
    plt.tight_layout()
    plt.show()

# Visualize overlay
visualize_forgery_overlay(forged_images, TRAIN_MASKS_PATH, n_samples=4)

<h2 style="text-align:center; color:#2c3e50;">
   ANALYZING MASKS
</h2>
<hr style="width:50%; margin:auto; height:2px; background-color:#2c3e50; border:none;">


In [ ]:
def analyze_masks(mask_files, masks_path):
    
    mask_stats = {
        'n_regions': [],
        'total_forged_pixels': [],
        'forged_percentage': [],
        'mask_shapes': []
    }
    
    print("Analyzing masks...")
    for mask_file in tqdm(mask_files[:100], desc="Processing masks"): 
        mask_path = os.path.join(masks_path, mask_file)
        mask = np.load(mask_path)
    
        mask_stats['mask_shapes'].append(mask.shape)
        
        if len(mask.shape) == 3:
            n_regions = mask.shape[0]
            combined_mask = np.max(mask, axis=0)
        else:
            n_regions = 1
            combined_mask = mask
        
        mask_stats['n_regions'].append(n_regions)
        
        total_pixels = combined_mask.shape[0] * combined_mask.shape[1]
        forged_pixels = np.sum(combined_mask > 0)
        forged_pct = (forged_pixels / total_pixels) * 100
        
        mask_stats['total_forged_pixels'].append(forged_pixels)
        mask_stats['forged_percentage'].append(forged_pct)
    
    return mask_stats

mask_stats = analyze_masks(mask_files, TRAIN_MASKS_PATH)

print(f"\n--- MASK STATISTICS ---")
print(f"Number of regions per image:")
print(f"  Min: {min(mask_stats['n_regions'])}")
print(f"  Max: {max(mask_stats['n_regions'])}")
print(f"  Mean: {np.mean(mask_stats['n_regions']):.2f}")
print(f"  Median: {np.median(mask_stats['n_regions']):.0f}")

print(f"\nForged pixels per image:")
print(f"  Min: {min(mask_stats['total_forged_pixels'])}")
print(f"  Max: {max(mask_stats['total_forged_pixels'])}")
print(f"  Mean: {np.mean(mask_stats['total_forged_pixels']):.0f}")
print(f"  Median: {np.median(mask_stats['total_forged_pixels']):.0f}")

print(f"\nForged area percentage:")
print(f"  Min: {min(mask_stats['forged_percentage']):.2f}%")
print(f"  Max: {max(mask_stats['forged_percentage']):.2f}%")
print(f"  Mean: {np.mean(mask_stats['forged_percentage']):.2f}%")
print(f"  Median: {np.median(mask_stats['forged_percentage']):.2f}%")

fig, axes = plt.subplots(1, 3, figsize=(18, 5))
fig.suptitle('Mask Statistics Distribution', fontsize=16, fontweight='bold')

axes[0].hist(mask_stats['n_regions'], bins=range(1, max(mask_stats['n_regions'])+2), 
             color='steelblue', edgecolor='black', alpha=0.7)
axes[0].set_title('Number of Forged Regions per Image')
axes[0].set_xlabel('Number of Regions')
axes[0].set_ylabel('Frequency')
axes[0].grid(True, alpha=0.3)

axes[1].hist(mask_stats['total_forged_pixels'], bins=30, 
             color='coral', edgecolor='black', alpha=0.7)
axes[1].set_title('Total Forged Pixels per Image')
axes[1].set_xlabel('Number of Pixels')
axes[1].set_ylabel('Frequency')
axes[1].grid(True, alpha=0.3)

axes[2].hist(mask_stats['forged_percentage'], bins=30, 
             color='mediumseagreen', edgecolor='black', alpha=0.7)
axes[2].set_title('Forged Area Percentage')
axes[2].set_xlabel('Percentage (%)')
axes[2].set_ylabel('Frequency')
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()


# DATA PREPARATION

**In this step, we organize the dataset into a structured DataFrame, combining image paths, mask paths, labels, and categories. We create a train-validation split while maintaining class balance, and implement helper functions to efficiently load and preprocess images and masks. Finally, we test the data loading pipeline by visualizing a few preprocessed samples along with their masks and overlayed forged regions, ensuring that our data preparation is ready for model training.**

In [ ]:
image_names = []
image_paths = []
mask_paths = []
labels = []  
categories = []

for img_name in authentic_images:
    image_names.append(img_name)
    image_paths.append(os.path.join(TRAIN_AUTHENTIC_PATH, img_name))
    mask_paths.append(None)  
    labels.append(0)
    categories.append('authentic')


for img_name in forged_images:
    image_names.append(img_name)
    image_paths.append(os.path.join(TRAIN_FORGED_PATH, img_name))
    
    mask_name = img_name.replace('.png', '.npy')
    mask_path = os.path.join(TRAIN_MASKS_PATH, mask_name)
    mask_paths.append(mask_path if os.path.exists(mask_path) else None)
    
    labels.append(1)
    categories.append('forged')

train_df = pd.DataFrame({
    'image_name': image_names,
    'image_path': image_paths,
    'mask_path': mask_paths,
    'label': labels,
    'category': categories
})

print(f"\nTraining DataFrame created!")
print(f"Total samples: {len(train_df)}")
print(f"\nClass distribution:")
print(train_df['category'].value_counts())
print(f"\nFirst few rows:")
print(train_df.head())

forged_df = train_df[train_df['category'] == 'forged']
missing_masks = forged_df['mask_path'].isna().sum()
print(f"\nForged images without masks: {missing_masks}")

In [ ]:
from sklearn.model_selection import train_test_split

train_data, val_data = train_test_split(
    train_df, 
    test_size=0.2, 
    random_state=42,
    stratify=train_df['label']
)

train_data = train_data.reset_index(drop=True)
val_data = val_data.reset_index(drop=True)

print(f"\nTrain set size: {len(train_data)}")
print(f"Validation set size: {len(val_data)}")

print(f"\nTrain set distribution:")
print(train_data['category'].value_counts())
print(f"  Forged percentage: {(train_data['label'].sum() / len(train_data)) * 100:.2f}%")

print(f"\nValidation set distribution:")
print(val_data['category'].value_counts())
print(f"  Forged percentage: {(val_data['label'].sum() / len(val_data)) * 100:.2f}%")

<h2 style="text-align:center; color:#2c3e50;">
  Creating Data Loading Helper Functions
</h2>
<hr style="width:50%; margin:auto; height:2px; background-color:#2c3e50; border:none;">


In [ ]:
def load_image(image_path, target_size=(512, 512)):
    img = Image.open(image_path).convert('RGB')
    img = img.resize(target_size, Image.BILINEAR)
    img_array = np.array(img, dtype=np.float32) / 255.0  # Normalize to [0, 1]
    return img_array

def load_mask(mask_path, target_size=(512, 512)):
    if mask_path is None or not os.path.exists(mask_path):
        
        return np.zeros(target_size[::-1], dtype=np.float32)
    
   
    mask = np.load(mask_path)
    

    if len(mask.shape) == 3:
        mask = np.max(mask, axis=0)
    
    mask_resized = cv2.resize(mask.astype(np.float32), target_size, interpolation=cv2.INTER_NEAREST)
    mask_binary = (mask_resized > 0.5).astype(np.float32)
    
    return mask_binary

def load_sample(df, idx, target_size=(512, 512)):
    row = df.iloc[idx]
    
    image = load_image(row['image_path'], target_size)
    mask = load_mask(row['mask_path'], target_size)
    label = row['label']
    
    return image, mask, label

print("Helper functions created:")
print("  - load_image(image_path, target_size)")
print("  - load_mask(mask_path, target_size)")
print("  - load_sample(df, idx, target_size)")

In [ ]:
TARGET_SIZE = (512, 512)

print(f"\nTesting data loading with target size: {TARGET_SIZE}")
print("Loading 3 samples from training set...\n")

fig, axes = plt.subplots(3, 3, figsize=(15, 15))
fig.suptitle(f'Preprocessed Samples (Resized to {TARGET_SIZE})', fontsize=16, fontweight='bold')

for i in range(3):
    forged_idx = train_data[train_data['category'] == 'forged'].index[i]
    image, mask, label = load_sample(train_data, forged_idx, TARGET_SIZE)
    
    axes[i, 0].imshow(image)
    axes[i, 0].set_title(f'Image (Label: {label})', fontsize=10)
    axes[i, 0].axis('off')
    
    axes[i, 1].imshow(mask, cmap='hot')
    axes[i, 1].set_title('Mask', fontsize=10)
    axes[i, 1].axis('off')
    overlay = (image * 255).astype(np.uint8)
    red_overlay = np.zeros_like(overlay)
    red_overlay[:,:,0] = (mask * 255).astype(np.uint8)
    blended = cv2.addWeighted(overlay, 0.6, red_overlay, 0.4, 0)
    
    axes[i, 2].imshow(blended)
    axes[i, 2].set_title('Overlay', fontsize=10)
    axes[i, 2].axis('off')

plt.tight_layout()
plt.show()

print("\nData loading test completed successfully!")
print(f"Image shape: {image.shape}")
print(f"Mask shape: {mask.shape}")
print(f"Image value range: [{image.min():.3f}, {image.max():.3f}]")
print(f"Mask unique values: {np.unique(mask)}")


# Data Augmentation

****In this stage, we prepare the dataset for model training by applying data augmentation and building the PyTorch pipeline. We create comprehensive augmentation strategies for training, including geometric, optical, intensity, and noise transformations, while keeping validation data normalized. A custom ForgeryDetectionDataset class is implemented to handle image and mask loading, resizing, combining multiple mask regions, and applying augmentations. Finally, we create train and validation dataloaders, test them, and visualize augmented samples with their masks and overlays to ensure the data pipeline works correctly.****

In [ ]:
print(f"\nPyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA device: {torch.cuda.get_device_name(0)}")
print("\n")

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}\n")


print("="*50)
print("CREATING AUGMENTATION PIPELINES")
print("="*50)

train_transform = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.5),
    A.Rotate(limit=30, p=0.5, border_mode=cv2.BORDER_CONSTANT, value=0),
    A.ShiftScaleRotate(
        shift_limit=0.1,
        scale_limit=0.15,
        rotate_limit=15,
        border_mode=cv2.BORDER_CONSTANT,
        value=0,
        p=0.5
    ),
    
    A.OneOf([
        A.OpticalDistortion(distort_limit=0.1, p=1.0),
        A.GridDistortion(num_steps=5, distort_limit=0.1, p=1.0),
        A.ElasticTransform(alpha=1, sigma=50, p=1.0),
    ], p=0.3),
    
    A.OneOf([
        A.RandomBrightnessContrast(
            brightness_limit=0.2,
            contrast_limit=0.2,
            p=1.0
        ),
        A.HueSaturationValue(
            hue_shift_limit=10,
            sat_shift_limit=20,
            val_shift_limit=20,
            p=1.0
        ),
        A.RandomGamma(gamma_limit=(80, 120), p=1.0),
    ], p=0.5),
    
    A.OneOf([
        A.GaussNoise(var_limit=(10.0, 50.0), p=1.0),
        A.GaussianBlur(blur_limit=(3, 5), p=1.0),
        A.MotionBlur(blur_limit=5, p=1.0),
    ], p=0.3),
    
    A.CLAHE(clip_limit=2.0, p=0.3),
    A.RandomShadow(
        shadow_roi=(0, 0.5, 1, 1),
        num_shadows_lower=1,
        num_shadows_upper=2,
        shadow_dimension=5,
        p=0.2
    ),
    
    A.Normalize(
        mean=[0.485, 0.456, 0.406],  
        std=[0.229, 0.224, 0.225],
        max_pixel_value=1.0,
    ),
    ToTensorV2(),
])

val_transform = A.Compose([
    A.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225],
        max_pixel_value=1.0,
    ),
    ToTensorV2(),
])

print("✓ Training augmentation pipeline created")
print("  - Geometric: Flip, Rotate, Shift, Scale")
print("  - Optical: Distortion, Grid, Elastic")
print("  - Intensity: Brightness, Contrast, HSV, Gamma")
print("  - Noise: Gaussian, Blur, Motion")
print("  - Other: CLAHE, Shadow")

print("\n✓ Validation augmentation pipeline created")
print("  - Only normalization (no augmentation)")
print("\n")


<h2 style="text-align:center; color:#2c3e50;">
  CREATING PYTORCH DATASET CLASS
</h2>
<hr style="width:50%; margin:auto; height:2px; background-color:#2c3e50; border:none;">


In [ ]:
class ForgeryDetectionDataset(Dataset):
    
    def __init__(self, dataframe, transform=None, target_size=(512, 512)):
    
        self.df = dataframe.reset_index(drop=True)
        self.transform = transform
        self.target_size = target_size
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        image_path = row['image_path']
        mask_path = row['mask_path']
        label = row['label']
        
        image = Image.open(image_path).convert('RGB')
        image = image.resize(self.target_size, Image.BILINEAR)
        image = np.array(image, dtype=np.float32) / 255.0
        
    
        if mask_path is not None and os.path.exists(mask_path):
            mask = np.load(mask_path)
            
            if len(mask.shape) == 3:
                mask = np.max(mask, axis=0)
            
            mask = cv2.resize(
                mask.astype(np.float32),
                self.target_size,
                interpolation=cv2.INTER_NEAREST
            )
            
        
            mask = (mask > 0.5).astype(np.float32)
        else:
        
            mask = np.zeros(self.target_size, dtype=np.float32)
        
      
        if self.transform:
            augmented = self.transform(image=image, mask=mask)
            image = augmented['image']
            mask = augmented['mask']
        else:
           
            image = torch.from_numpy(image).permute(2, 0, 1) 
            mask = torch.from_numpy(mask).unsqueeze(0) 
        
        if len(mask.shape) == 2:
            mask = mask.unsqueeze(0)
        
        return {
            'image': image,
            'mask': mask,
            'label': torch.tensor(label, dtype=torch.float32)
        }



<h2 style="text-align:center; color:#2c3e50;">
  CCREATING DATALOADERS
</h2>
<hr style="width:50%; margin:auto; height:2px; background-color:#2c3e50; border:none;">


In [ ]:
BATCH_SIZE = 6
NUM_WORKERS = 2
TARGET_SIZE = (512, 512)

train_dataset = ForgeryDetectionDataset(
    dataframe=train_data,
    transform=train_transform,
    target_size=TARGET_SIZE
)

val_dataset = ForgeryDetectionDataset(
    dataframe=val_data,
    transform=val_transform,
    target_size=TARGET_SIZE
)

train_loader = DataLoader(
    train_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=NUM_WORKERS,
    pin_memory=True if torch.cuda.is_available() else False,
    drop_last=True
)

val_loader = DataLoader(
    val_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=NUM_WORKERS,
    pin_memory=True if torch.cuda.is_available() else False
)

print(f"✓ Dataloaders created")
print(f"  - Batch size: {BATCH_SIZE}")
print(f"  - Training batches: {len(train_loader)}")
print(f"  - Validation batches: {len(val_loader)}")
print(f"  - Training samples: {len(train_dataset)}")
print(f"  - Validation samples: {len(val_dataset)}")
print("\n")

In [ ]:
sample_batch = next(iter(train_loader))

print(f"Sample batch shapes:")
print(f"  - Images: {sample_batch['image'].shape}")
print(f"  - Masks: {sample_batch['mask'].shape}")
print(f"  - Labels: {sample_batch['label'].shape}")

print(f"\nImage tensor stats:")
print(f"  - Min: {sample_batch['image'].min():.3f}")
print(f"  - Max: {sample_batch['image'].max():.3f}")
print(f"  - Mean: {sample_batch['image'].mean():.3f}")

print(f"\nMask tensor stats:")
print(f"  - Unique values: {torch.unique(sample_batch['mask'])}")
print(f"  - Forged pixels ratio: {sample_batch['mask'].mean():.4f}")


def denormalize(image):
    mean = torch.tensor([0.485, 0.456, 0.406]).view(3, 1, 1)
    std = torch.tensor([0.229, 0.224, 0.225]).view(3, 1, 1)
    image = image * std + mean
    return torch.clamp(image, 0, 1)

fig, axes = plt.subplots(3, 3, figsize=(15, 15))
fig.suptitle('Augmented Training Samples', fontsize=16, fontweight='bold')

for i in range(3):
    # Get sample
    img = denormalize(sample_batch['image'][i]).permute(1, 2, 0).cpu().numpy()
    mask = sample_batch['mask'][i, 0].cpu().numpy()
    label = sample_batch['label'][i].item()
    
    # Plot
    axes[i, 0].imshow(img)
    axes[i, 0].set_title(f'Augmented Image (Label: {label:.0f})')
    axes[i, 0].axis('off')
    
    axes[i, 1].imshow(mask, cmap='hot')
    axes[i, 1].set_title('Augmented Mask')
    axes[i, 1].axis('off')
    
    # Overlay
    overlay = (img * 255).astype(np.uint8)
    red_overlay = np.zeros_like(overlay)
    red_overlay[:,:,0] = (mask * 255).astype(np.uint8)
    blended = cv2.addWeighted(overlay, 0.6, red_overlay, 0.4, 0)
    
    axes[i, 2].imshow(blended)
    axes[i, 2].set_title('Overlay')
    axes[i, 2].axis('off')

plt.tight_layout()
plt.show()



# U-NET MODEL ARCHITECTURE

In this step, we define the core U-Net model for forgery detection, based on a configurable encoder backbone. We set up a custom loss strategy combining Binary Cross-Entropy and Dice loss . The model is then prepared with an AdamW optimizer and a learning rate scheduler to adapt during training. Finally, we define evaluation metricsDice coefficient, IoU, and pixel accuracyto quantitatively measure how well the model predicts forged regions. This setup ensures the model is ready for robust training and precise evaluation.

In [ ]:
class UNetForgeryDetector(nn.Module):
    def __init__(
        self,
        encoder_name='resnet34',
        encoder_weights=None,   
        in_channels=3,
        classes=1,
        activation=None
    ):
        super(UNetForgeryDetector, self).__init__()
        
        self.model = smp.Unet(
            encoder_name=encoder_name,
            encoder_weights=encoder_weights, 
            in_channels=in_channels,
            classes=classes,
            activation=activation
        )
    
    def forward(self, x):
        return self.model(x)



MODEL_NAME = 'resnet34'  
ENCODER_WEIGHTS = None  

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = UNetForgeryDetector(
    encoder_name=MODEL_NAME,
    encoder_weights=ENCODER_WEIGHTS,
    in_channels=3,
    classes=1,
    activation=None
)

model = model.to(device)

print(f"✓ U-Net model created")
print(f"  - Encoder: {MODEL_NAME}")
print(f"  - Pre-trained weights: {ENCODER_WEIGHTS}")
print(f"  - Input channels: 3 (RGB)")
print(f"  - Output classes: 1 (binary segmentation)")
print(f"  - Device: {device}")
print("\n")


def count_parameters(model):
    total = sum(p.numel() for p in model.parameters())
    trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)
    return total, trainable

total_params, trainable_params = count_parameters(model)

print(f"Model Parameters:")
print(f"  - Total: {total_params:,}")
print(f"  - Trainable: {trainable_params:,}")
print(f"  - Size: ~{total_params * 4 / (1024**2):.2f} MB (FP32)")


print("\nTesting forward pass...")
with torch.no_grad():
    dummy_input = torch.randn(2, 3, 512, 512).to(device)
    dummy_output = model(dummy_input)
    print(f"  - Input shape: {dummy_input.shape}")
    print(f"  - Output shape: {dummy_output.shape}")
    print("✓ Forward pass successful!")
print("\n")



print("="*60)
print(" DEFINING LOSS FUNCTIONS")
print("="*60)

class DiceLoss(nn.Module):  
    def __init__(self, smooth=1.0):
        super(DiceLoss, self).__init__()
        self.smooth = smooth
    
    def forward(self, predictions, targets):
        predictions = torch.sigmoid(predictions)
        
        predictions = predictions.view(-1)
        targets = targets.view(-1)
        
        intersection = (predictions * targets).sum()
        dice = (2. * intersection + self.smooth) / (
            predictions.sum() + targets.sum() + self.smooth
        )
        
        return 1 - dice

class CombinedLoss(nn.Module):
    
    def __init__(self, bce_weight=0.5, dice_weight=0.5):
        super(CombinedLoss, self).__init__()
        self.bce_weight = bce_weight
        self.dice_weight = dice_weight
        self.bce = nn.BCEWithLogitsLoss()
        self.dice = DiceLoss()
    
    def forward(self, predictions, targets):
        bce_loss = self.bce(predictions, targets)
        dice_loss = self.dice(predictions, targets)
        
        combined = self.bce_weight * bce_loss + self.dice_weight * dice_loss
        
        return combined, bce_loss, dice_loss

criterion = CombinedLoss(bce_weight=0.5, dice_weight=0.5)

print("✓ Loss functions created")
print("  - BCE Loss: Binary Cross-Entropy with Logits")
print("  - Dice Loss: Custom implementation")
print("  - Combined Loss: 0.5 * BCE + 0.5 * Dice")
print("\n")


print("="*60)
print(" CONFIGURING OPTIMIZER AND SCHEDULER")
print("="*60)

LEARNING_RATE = 3e-4
WEIGHT_DECAY = 1e-5

optimizer = torch.optim.AdamW(
    model.parameters(),
    lr=LEARNING_RATE,
    weight_decay=WEIGHT_DECAY
)


scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer,
    mode='min',
    factor=0.5,
    patience=5,
    verbose=True,
    min_lr=1e-7
)

print(f"✓ Optimizer: AdamW")
print(f"  - Learning rate: {LEARNING_RATE}")
print(f"  - Weight decay: {WEIGHT_DECAY}")

print(f"\n✓ Scheduler: ReduceLROnPlateau")
print(f"  - Mode: min (reduce on plateau)")
print(f"  - Factor: 0.5 (halve LR)")
print(f"  - Patience: 5 epochs")
print(f"  - Min LR: 1e-7")
print("\n")

print("="*60)
print("DEFINING EVALUATION METRICS")
print("="*60)

def dice_coefficient(predictions, targets, threshold=0.5, smooth=1.0):

    predictions = (torch.sigmoid(predictions) > threshold).float()
    predictions = predictions.view(-1)
    targets = targets.view(-1)
    
    intersection = (predictions * targets).sum()
    dice = (2. * intersection + smooth) / (
        predictions.sum() + targets.sum() + smooth
    )
    
    return dice.item()

def iou_score(predictions, targets, threshold=0.5, smooth=1.0):
    predictions = (torch.sigmoid(predictions) > threshold).float()
    predictions = predictions.view(-1)
    targets = targets.view(-1)
    
    intersection = (predictions * targets).sum()
    union = predictions.sum() + targets.sum() - intersection
    
    iou = (intersection + smooth) / (union + smooth)
    
    return iou.item()

def pixel_accuracy(predictions, targets, threshold=0.5):
   
    predictions = (torch.sigmoid(predictions) > threshold).float()
    correct = (predictions == targets).float().sum()
    total = targets.numel()
    
    return (correct / total).item()

print("✓ Evaluation metrics defined")
print("  - Dice Coefficient")
print("  - IoU (Intersection over Union)")
print("  - Pixel Accuracy")
print("\n")

# Training Configuration 

**This section sets up the training workflow for the U-Net forgery detection model. We define a configuration dictionary specifying the number of epochs, early stopping patience, checkpoint settings, and the metric to track the best model. Two utility classes are created: AverageMeter to track running averages of loss and metrics during training, and EarlyStopping to halt training if the model stops improving.**

**We then define train_one_epoch() and validate() functions to handle the training and validation loops, including forward passes, loss computation, metric calculation (Dice, IoU), backpropagation, and optimizer updates. Progress is displayed using a tqdm progress bar, with real-time reporting of loss and evaluation metrics. This setup ensures structured, monitored, and efficient model training.**

In [ ]:
CONFIG = {
    'epochs': 1,
    'early_stopping_patience': 7,
    'best_model_metric': 'val_dice',  
    'checkpoint_dir': 'checkpoints',
    'save_best_only': True,
}

print(f"""
Training Configuration:
  - Epochs: {CONFIG['epochs']}
  - Early stopping patience: {CONFIG['early_stopping_patience']}
  - Best model metric: {CONFIG['best_model_metric']}
  - Checkpoint directory: {CONFIG['checkpoint_dir']}
  - Save best only: {CONFIG['save_best_only']}
""")

os.makedirs(CONFIG['checkpoint_dir'], exist_ok=True)


class AverageMeter:

    def __init__(self):
        self.reset()
    
    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0
    
    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

class EarlyStopping:
    def __init__(self, patience=7, mode='min', delta=0.0001):
        self.patience = patience
        self.mode = mode
        self.delta = delta
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        
    def __call__(self, score):
        if self.best_score is None:
            self.best_score = score
            return False
        
        if self.mode == 'min':
            improved = score < (self.best_score - self.delta)
        else:
            improved = score > (self.best_score + self.delta)
        
        if improved:
            self.best_score = score
            self.counter = 0
        else:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True
        
        return self.early_stop

print("✓ Utility classes created:")
print("  - AverageMeter: Track running averages")
print("  - EarlyStopping: Stop training early if no improvement")
print("\n")


def train_one_epoch(model, dataloader, criterion, optimizer, device, epoch):
    model.train()
    
    loss_meter = AverageMeter()
    bce_meter = AverageMeter()
    dice_loss_meter = AverageMeter()
    dice_score_meter = AverageMeter()
    iou_meter = AverageMeter()
    
    pbar = tqdm(dataloader, desc=f'Epoch {epoch} [TRAIN]')
    
    for batch_idx, batch in enumerate(pbar):
        
        images = batch['image'].to(device)
        masks = batch['mask'].to(device)
        
        outputs = model(images)
        
        loss, bce_loss, dice_loss = criterion(outputs, masks)
        
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        with torch.no_grad():
            dice_score = dice_coefficient(outputs, masks)
            iou = iou_score(outputs, masks)
        
        batch_size = images.size(0)
        loss_meter.update(loss.item(), batch_size)
        bce_meter.update(bce_loss.item(), batch_size)
        dice_loss_meter.update(dice_loss.item(), batch_size)
        dice_score_meter.update(dice_score, batch_size)
        iou_meter.update(iou, batch_size)
        
        pbar.set_postfix({
            'loss': f'{loss_meter.avg:.4f}',
            'dice': f'{dice_score_meter.avg:.4f}',
            'iou': f'{iou_meter.avg:.4f}'
        })
    
    return {
        'loss': loss_meter.avg,
        'bce_loss': bce_meter.avg,
        'dice_loss': dice_loss_meter.avg,
        'dice_score': dice_score_meter.avg,
        'iou': iou_meter.avg
    }

def validate(model, dataloader, criterion, device, epoch):
    model.eval()
    
    loss_meter = AverageMeter()
    bce_meter = AverageMeter()
    dice_loss_meter = AverageMeter()
    dice_score_meter = AverageMeter()
    iou_meter = AverageMeter()
    
    pbar = tqdm(dataloader, desc=f'Epoch {epoch} [VALID]')
    
    with torch.no_grad():
        for batch_idx, batch in enumerate(pbar):
            
            images = batch['image'].to(device)
            masks = batch['mask'].to(device)
            
            outputs = model(images)
            
            loss, bce_loss, dice_loss = criterion(outputs, masks)
            
            dice_score = dice_coefficient(outputs, masks)
            iou = iou_score(outputs, masks)
            
            batch_size = images.size(0)
            loss_meter.update(loss.item(), batch_size)
            bce_meter.update(bce_loss.item(), batch_size)
            dice_loss_meter.update(dice_loss.item(), batch_size)
            dice_score_meter.update(dice_score, batch_size)
            iou_meter.update(iou, batch_size)
            
            pbar.set_postfix({
                'loss': f'{loss_meter.avg:.4f}',
                'dice': f'{dice_score_meter.avg:.4f}',
                'iou': f'{iou_meter.avg:.4f}'
            })
    
    return {
        'loss': loss_meter.avg,
        'bce_loss': bce_meter.avg,
        'dice_loss': dice_loss_meter.avg,
        'dice_score': dice_score_meter.avg,
        'iou': iou_meter.avg
    }



# MODEL TRAINING

<h2>Training Loop Overview</h2>

<p>This section runs the <strong>full training loop</strong> for the U-Net forgery detection model. Key steps include:</p>

<h3>1. Setup</h3>
<ul>
  <li>Initialize <code>history</code> to track metrics over epochs.</li>
  <li>Save the best model weights (<code>best_model_wts</code>) based on validation Dice score.</li>
  <li>Instantiate <code>EarlyStopping</code> to halt training if validation performance does not improve.</li>
</ul>

<h3>2. Training Loop</h3>
<ul>
  <li>Iterate over the specified number of epochs.</li>
  <li>For each epoch:
    <ul>
      <li>Train on the training dataset using <code>train_one_epoch</code> and validate using <code>validate</code>.</li>
      <li>Update the learning rate using <code>ReduceLROnPlateau</code> scheduler.</li>
      <li>Track metrics: loss, BCE loss, Dice loss, Dice score, IoU.</li>
      <li>Save a checkpoint if a new best validation Dice score is achieved.</li>
      <li>Apply early stopping if no improvement for the configured patience.</li>
    </ul>
  </li>
</ul>

<h3>3. Post-training</h3>
<ul>
  <li>Load the best model weights into the model.</li>
  <li>Convert the training <code>history</code> into a DataFrame and visualize:
    <ul>
      <li>Loss, Dice score, IoU curves over epochs.</li>
      <li>Learning rate schedule.</li>
      <li>Train vs validation comparisons to detect trends and overfitting.</li>
    </ul>
  </li>
  <li>Identify the <strong>best epoch</strong> and print all relevant metrics.</li>
  <li>Analyze overfitting by comparing train vs validation Dice and loss gaps, with warnings if the gap indicates potential overfitting.</li>
</ul>

<p>This ensures structured monitoring, automatic checkpointing, and performance evaluation for optimal model training.</p>


In [ ]:
history = defaultdict(list)
best_dice = 0.0
best_model_wts = copy.deepcopy(model.state_dict())

early_stopping = EarlyStopping(
    patience=CONFIG['early_stopping_patience'],
    mode='max',  
    delta=0.001
)

training_start_time = time.time()

print(f"Starting training for {CONFIG['epochs']} epochs...")
print(f"Training samples: {len(train_dataset)}")
print(f"Validation samples: {len(val_dataset)}")
print(f"Device: {device}")
print("="*80)
print("\n")


for epoch in range(1, CONFIG['epochs'] + 1):
    epoch_start_time = time.time()
    
    print(f"\n{'='*80}")
    print(f"EPOCH {epoch}/{CONFIG['epochs']}")
    print(f"{'='*80}")
    
    train_metrics = train_one_epoch(
        model=model,
        dataloader=train_loader,
        criterion=criterion,
        optimizer=optimizer,
        device=device,
        epoch=epoch
    )
    
    val_metrics = validate(
        model=model,
        dataloader=val_loader,
        criterion=criterion,
        device=device,
        epoch=epoch
    )
    
    scheduler.step(val_metrics['loss'])
    
    current_lr = optimizer.param_groups[0]['lr']
    
    epoch_time = time.time() - epoch_start_time
    
    print(f"\n{'='*80}")
    print(f"EPOCH {epoch} SUMMARY")
    print(f"{'='*80}")
    print(f"Time: {epoch_time:.2f}s | LR: {current_lr:.2e}")
    print(f"\nTrain Metrics:")
    print(f"  Loss: {train_metrics['loss']:.4f} | BCE: {train_metrics['bce_loss']:.4f} | Dice Loss: {train_metrics['dice_loss']:.4f}")
    print(f"  Dice Score: {train_metrics['dice_score']:.4f} | IoU: {train_metrics['iou']:.4f}")
    print(f"\nValidation Metrics:")
    print(f"  Loss: {val_metrics['loss']:.4f} | BCE: {val_metrics['bce_loss']:.4f} | Dice Loss: {val_metrics['dice_loss']:.4f}")
    print(f"  Dice Score: {val_metrics['dice_score']:.4f} | IoU: {val_metrics['iou']:.4f}")
    
    history['epoch'].append(epoch)
    history['lr'].append(current_lr)
    history['train_loss'].append(train_metrics['loss'])
    history['train_dice'].append(train_metrics['dice_score'])
    history['train_iou'].append(train_metrics['iou'])
    history['val_loss'].append(val_metrics['loss'])
    history['val_dice'].append(val_metrics['dice_score'])
    history['val_iou'].append(val_metrics['iou'])
    
    if val_metrics['dice_score'] > best_dice:
        best_dice = val_metrics['dice_score']
        best_model_wts = copy.deepcopy(model.state_dict())
        
        checkpoint_path = os.path.join(
            CONFIG['checkpoint_dir'],
            f'best_model_epoch{epoch}_dice{best_dice:.4f}.pth'
        )
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'scheduler_state_dict': scheduler.state_dict(),
            'best_dice': best_dice,
            'val_metrics': val_metrics,
        }, checkpoint_path)
        
        print(f"\n🎯 New best model saved! Dice: {best_dice:.4f}")
        print(f"   Saved to: {checkpoint_path}")
    
    if early_stopping(val_metrics['dice_score']):
        print(f"\n  Early stopping triggered after {epoch} epochs")
        print(f"   No improvement for {CONFIG['early_stopping_patience']} epochs")
        break
    
    print(f"{'='*80}\n")



training_time = time.time() - training_start_time

print("\n")
print("="*80)
print("TRAINING COMPLETED!")
print("="*80)
print(f"Total training time: {training_time/60:.2f} minutes")
print(f"Best validation Dice score: {best_dice:.4f}")
print(f"Total epochs trained: {len(history['epoch'])}")
print("="*80)
print("\n")


model.load_state_dict(best_model_wts)
print("✓ Best model weights loaded into model")
print("\n")


history_df = pd.DataFrame(history)

fig, axes = plt.subplots(2, 3, figsize=(20, 12))
fig.suptitle('Training History', fontsize=16, fontweight='bold')

axes[0, 0].plot(history_df['epoch'], history_df['train_loss'], 
                label='Train Loss', marker='o', linewidth=2)
axes[0, 0].plot(history_df['epoch'], history_df['val_loss'], 
                label='Val Loss', marker='s', linewidth=2)
axes[0, 0].set_xlabel('Epoch')
axes[0, 0].set_ylabel('Loss')
axes[0, 0].set_title('Loss Curve')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

axes[0, 1].plot(history_df['epoch'], history_df['train_dice'], 
                label='Train Dice', marker='o', linewidth=2, color='green')
axes[0, 1].plot(history_df['epoch'], history_df['val_dice'], 
                label='Val Dice', marker='s', linewidth=2, color='orange')
axes[0, 1].set_xlabel('Epoch')
axes[0, 1].set_ylabel('Dice Score')
axes[0, 1].set_title('Dice Score Curve')
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3)

axes[0, 2].plot(history_df['epoch'], history_df['train_iou'], 
                label='Train IoU', marker='o', linewidth=2, color='purple')
axes[0, 2].plot(history_df['epoch'], history_df['val_iou'], 
                label='Val IoU', marker='s', linewidth=2, color='brown')
axes[0, 2].set_xlabel('Epoch')
axes[0, 2].set_ylabel('IoU Score')
axes[0, 2].set_title('IoU Score Curve')
axes[0, 2].legend()
axes[0, 2].grid(True, alpha=0.3)

axes[1, 0].plot(history_df['epoch'], history_df['lr'], 
                marker='o', linewidth=2, color='red')
axes[1, 0].set_xlabel('Epoch')
axes[1, 0].set_ylabel('Learning Rate')
axes[1, 0].set_title('Learning Rate Schedule')
axes[1, 0].set_yscale('log')
axes[1, 0].grid(True, alpha=0.3)


axes[1, 1].scatter(history_df['train_loss'], history_df['val_loss'], 
                   c=history_df['epoch'], cmap='viridis', s=100, alpha=0.6)
axes[1, 1].plot([history_df['train_loss'].min(), history_df['train_loss'].max()],
                [history_df['train_loss'].min(), history_df['train_loss'].max()],
                'r--', linewidth=2, label='Perfect fit')
axes[1, 1].set_xlabel('Train Loss')
axes[1, 1].set_ylabel('Val Loss')
axes[1, 1].set_title('Train vs Val Loss')
axes[1, 1].legend()
axes[1, 1].grid(True, alpha=0.3)

axes[1, 2].scatter(history_df['train_dice'], history_df['val_dice'], 
                   c=history_df['epoch'], cmap='viridis', s=100, alpha=0.6)
axes[1, 2].plot([history_df['train_dice'].min(), history_df['train_dice'].max()],
                [history_df['train_dice'].min(), history_df['train_dice'].max()],
                'r--', linewidth=2, label='Perfect fit')
axes[1, 2].set_xlabel('Train Dice')
axes[1, 2].set_ylabel('Val Dice')
axes[1, 2].set_title('Train vs Val Dice')
axes[1, 2].legend()
axes[1, 2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("✓ Training curves plotted")
print("\n")



best_epoch_idx = history_df['val_dice'].idxmax()
best_epoch_data = history_df.iloc[best_epoch_idx]

print(f"\nBest Epoch: {int(best_epoch_data['epoch'])}")
print(f"\nMetrics at Best Epoch:")
print(f"  Train Loss: {best_epoch_data['train_loss']:.4f}")
print(f"  Val Loss: {best_epoch_data['val_loss']:.4f}")
print(f"  Train Dice: {best_epoch_data['train_dice']:.4f}")
print(f"  Val Dice: {best_epoch_data['val_dice']:.4f}")
print(f"  Train IoU: {best_epoch_data['train_iou']:.4f}")
print(f"  Val IoU: {best_epoch_data['val_iou']:.4f}")
print(f"  Learning Rate: {best_epoch_data['lr']:.2e}")

# Check for overfitting
dice_gap = best_epoch_data['train_dice'] - best_epoch_data['val_dice']
loss_gap = best_epoch_data['val_loss'] - best_epoch_data['train_loss']

print(f"\nOverfitting Analysis:")
print(f"  Dice gap (Train - Val): {dice_gap:.4f}")
print(f"  Loss gap (Val - Train): {loss_gap:.4f}")

if dice_gap > 0.1:
    print("  ⚠️  Warning: Significant overfitting detected (Dice gap > 0.1)")
elif dice_gap > 0.05:
    print("  ⚡ Moderate overfitting (Dice gap > 0.05)")
else:
    print("  ✓ Good generalization (Dice gap < 0.05)")

print("\n")



# Validation

In [ ]:
def predict_batch(model, images, device, threshold=0.5):
    model.eval()
    with torch.no_grad():
        outputs = model(images.to(device))
        predictions = torch.sigmoid(outputs)
        binary_predictions = (predictions > threshold).float()
    return predictions, binary_predictions

def visualize_predictions(model, dataloader, device, n_samples=6, threshold=0.5):
  
    model.eval()
    
    fig, axes = plt.subplots(n_samples, 4, figsize=(20, 5*n_samples))
    fig.suptitle(f'Validation Predictions (Threshold: {threshold})', 
                 fontsize=16, fontweight='bold')
    
    batch = next(iter(dataloader))
    images = batch['image'][:n_samples].to(device)
    masks = batch['mask'][:n_samples]
    labels = batch['label'][:n_samples]
    
    with torch.no_grad():
        outputs = model(images)
        predictions = torch.sigmoid(outputs)
        binary_preds = (predictions > threshold).float()
    
    def denormalize(img):
        mean = torch.tensor([0.485, 0.456, 0.406]).view(3, 1, 1).to(img.device)
        std = torch.tensor([0.229, 0.224, 0.225]).view(3, 1, 1).to(img.device)
        return torch.clamp(img * std + mean, 0, 1)
    
    for i in range(n_samples):
        img = denormalize(images[i]).cpu().permute(1, 2, 0).numpy()
        axes[i, 0].imshow(img)
        axes[i, 0].set_title(f'Image (Label: {labels[i].item():.0f})')
        axes[i, 0].axis('off')
        
        gt_mask = masks[i, 0].cpu().numpy()
        axes[i, 1].imshow(gt_mask, cmap='hot', vmin=0, vmax=1)
        axes[i, 1].set_title('Ground Truth Mask')
        axes[i, 1].axis('off')
        
        pred_prob = predictions[i, 0].cpu().numpy()
        axes[i, 2].imshow(pred_prob, cmap='hot', vmin=0, vmax=1)
        axes[i, 2].set_title(f'Predicted (Prob)\nMax: {pred_prob.max():.3f}')
        axes[i, 2].axis('off')
        
        pred_binary = binary_preds[i, 0].cpu().numpy()
        axes[i, 3].imshow(pred_binary, cmap='hot', vmin=0, vmax=1)
        
        # Calculate metrics for this sample
        dice = dice_coefficient(
            outputs[i:i+1], 
            masks[i:i+1].to(device), 
            threshold=threshold
        )
        axes[i, 3].set_title(f'Binary Prediction\nDice: {dice:.3f}')
        axes[i, 3].axis('off')
    
    plt.tight_layout()
    plt.show()

# Visualize predictions
print("\nVisualizing predictions on validation set...")
visualize_predictions(
    model=model,
    dataloader=val_loader,
    device=device,
    n_samples=6,
    threshold=0.5
)

print("✓ Validation predictions visualized")
print("\n")


print("="*80)
print("STEP 7.1: OPTIMIZING PREDICTION THRESHOLD")
print("="*80)

def evaluate_threshold(model, dataloader, device, threshold):
    model.eval()
    dice_scores = []
    iou_scores = []
    
    with torch.no_grad():
        for batch in dataloader:
            images = batch['image'].to(device)
            masks = batch['mask'].to(device)
            
            outputs = model(images)
            
            dice = dice_coefficient(outputs, masks, threshold=threshold)
            iou = iou_score(outputs, masks, threshold=threshold)
            
            dice_scores.append(dice)
            iou_scores.append(iou)
    
    return np.mean(dice_scores), np.mean(iou_scores)

print("Testing different thresholds...")
thresholds = [0.3,  0.7]
threshold_results = []

for thresh in tqdm(thresholds, desc="Testing thresholds"):
    dice, iou = evaluate_threshold(model, val_loader, device, thresh)
    threshold_results.append({
        'threshold': thresh,
        'dice': dice,
        'iou': iou
    })
    print(f"  Threshold {thresh:.2f}: Dice={dice:.4f}, IoU={iou:.4f}")

threshold_df = pd.DataFrame(threshold_results)
best_threshold_idx = threshold_df['dice'].idxmax()
best_threshold = threshold_df.iloc[best_threshold_idx]['threshold']
best_dice = threshold_df.iloc[best_threshold_idx]['dice']

print(f"\n✓ Best threshold: {best_threshold}")
print(f"  Dice score: {best_dice:.4f}")

fig, axes = plt.subplots(1, 2, figsize=(15, 5))
fig.suptitle('Threshold Optimization', fontsize=16, fontweight='bold')

axes[0].plot(threshold_df['threshold'], threshold_df['dice'], 
             marker='o', linewidth=2, markersize=8, color='green')
axes[0].axvline(best_threshold, color='red', linestyle='--', 
                label=f'Best: {best_threshold}')
axes[0].set_xlabel('Threshold')
axes[0].set_ylabel('Dice Score')
axes[0].set_title('Dice Score vs Threshold')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

axes[1].plot(threshold_df['threshold'], threshold_df['iou'], 
             marker='s', linewidth=2, markersize=8, color='purple')
axes[1].axvline(best_threshold, color='red', linestyle='--', 
                label=f'Best: {best_threshold}')
axes[1].set_xlabel('Threshold')
axes[1].set_ylabel('IoU Score')
axes[1].set_title('IoU Score vs Threshold')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n")

# POST-PROCESSING

In [ ]:
def remove_small_regions(mask, min_size=100):

    from scipy import ndimage
    
    labeled_mask, num_features = ndimage.label(mask)
    
    for region_label in range(1, num_features + 1):
        region = (labeled_mask == region_label)
        if region.sum() < min_size:
            mask[region] = 0
    
    return mask

def apply_morphology(mask, kernel_size=3):
   
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (kernel_size, kernel_size))
    
    # Opening: remove small noise
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)
    
    # Closing: fill small holes
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)
    
    return mask

def post_process_mask(mask, threshold=0.5, min_size=100, morphology_kernel=3):

    # Binarize
    binary_mask = (mask > threshold).astype(np.uint8)
    
    # Apply morphology
    if morphology_kernel > 0:
        binary_mask = apply_morphology(binary_mask, morphology_kernel)
    
    # Remove small regions
    if min_size > 0:
        binary_mask = remove_small_regions(binary_mask, min_size)
    
    return binary_mask

print("✓ Post-processing functions created:")
print("  - remove_small_regions(): Remove small isolated regions")
print("  - apply_morphology(): Morphological opening/closing")
print("  - post_process_mask(): Complete pipeline")
print("\n")


print("="*80)
print("STEP 8.1: RUN LENGTH ENCODING (RLE) FUNCTIONS")
print("="*80)

def rle_encode(mask):
    
    pixels = mask.T.flatten()
    
    pixels = np.concatenate([[0], pixels, [0]])
    
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    
    return ' '.join(str(x) for x in runs)

def rle_decode(rle_string, shape):
    if rle_string == "authentic":
        return np.zeros(shape, dtype=np.uint8)
    
    runs = np.array([int(x) for x in rle_string.split()])
    starts = runs[::2]
    lengths = runs[1::2]
    
    mask = np.zeros(shape[0] * shape[1], dtype=np.uint8)
    for start, length in zip(starts, lengths):
        mask[start:start+length] = 1

    mask = mask.reshape(shape[::-1]).T
    
    return mask

print("✓ RLE encoding functions created:")
print("  - rle_encode(): Convert mask to RLE string")
print("  - rle_decode(): Convert RLE string back to mask")

print("\nTesting RLE encoding...")
test_mask = np.zeros((10, 10), dtype=np.uint8)
test_mask[2:5, 3:7] = 1
test_rle = rle_encode(test_mask)
test_decoded = rle_decode(test_rle, test_mask.shape)
print(f"  Original mask sum: {test_mask.sum()}")
print(f"  RLE string: {test_rle}")
print(f"  Decoded mask sum: {test_decoded.sum()}")
print(f"  Encoding correct: {np.array_equal(test_mask, test_decoded)}")
print("\n")


# GENERATE TEST PREDICTIONS

In [ ]:
import os
import numpy as np
import pandas as pd
from PIL import Image
from tqdm import tqdm
import torch
import cv2

# -----------------------
# CONFIGURATION
# -----------------------
BASE_PATH = "/kaggle/input/recodai-luc-scientific-image-forgery-detection"
TEST_IMAGES_PATH = os.path.join(BASE_PATH, "test_images")
SAMPLE_SUBMISSION_PATH = os.path.join(BASE_PATH, "sample_submission.csv")

PREDICTION_CONFIG = {
    'threshold': best_threshold,       # à définir selon ton entraînement
    'min_region_size': 50,
    'morphology_kernel': 3,
    'min_forgery_pixels': 100,
}

# -----------------------
# FONCTION DE PREDICTION
# -----------------------
def predict_test_image(model, image_path, device, config, target_size=(512, 512)):
    model.eval()
    
    image = Image.open(image_path).convert('RGB')
    original_size = image.size
    image = image.resize(target_size, Image.BILINEAR)
    image_array = np.array(image, dtype=np.float32) / 255.0

    transformed = val_transform(image=image_array, mask=np.zeros(target_size))
    image_tensor = transformed['image'].unsqueeze(0).to(device)
    
    with torch.no_grad():
        output = model(image_tensor)
        prediction = torch.sigmoid(output)[0, 0].cpu().numpy()
    
    binary_mask = post_process_mask(
        prediction,
        threshold=config['threshold'],
        min_size=config['min_region_size'],
        morphology_kernel=config['morphology_kernel']
    )
    
    binary_mask = cv2.resize(
        binary_mask.astype(np.float32),
        original_size,
        interpolation=cv2.INTER_NEAREST
    ).astype(np.uint8)

    total_forged_pixels = binary_mask.sum()
    
    if total_forged_pixels < config['min_forgery_pixels']:
        return "authentic"
    else:
        rle = rle_encode(binary_mask)
        if isinstance(rle, (list, np.ndarray)):
            rle = "[" + " ".join(map(str, rle)) + "]"
        elif not (rle.startswith("[") and rle.endswith("]")):
            rle = "[" + rle.strip() + "]"
        return rle

# -----------------------
# GENERER LES PREDICTIONS POUR LES IMAGES VISIBLES
# -----------------------
test_images = sorted([f for f in os.listdir(TEST_IMAGES_PATH) if f.endswith('.png')])
predictions = {}

for img_name in tqdm(test_images, desc="Processing test images"):
    case_id = img_name.replace('.png', '')
    img_path = os.path.join(TEST_IMAGES_PATH, img_name)
    
    rle_or_authentic = predict_test_image(model, img_path, device, PREDICTION_CONFIG)
    predictions[case_id] = rle_or_authentic

# -----------------------
# UTILISER LE SAMPLE SUBMISSION POUR TOUTES LES LIGNES
# -----------------------
sample_submission = pd.read_csv(SAMPLE_SUBMISSION_PATH)
submission_data = []

for case_id in sample_submission['case_id']:
    case_id_str = str(case_id)
    if case_id_str in predictions:
        annotation = predictions[case_id_str]
    else:
        # Pour les images invisibles du test privé, mettre "authentic" par défaut
        annotation = "authentic"
    submission_data.append({'case_id': case_id, 'annotation': annotation})

submission_df = pd.DataFrame(submission_data)

# -----------------------
# CHECK FINAL
# -----------------------
assert submission_df.shape[1] == 2
assert submission_df.columns.tolist() == ['case_id', 'annotation']
assert submission_df['case_id'].apply(lambda x: isinstance(x, (int, np.integer))).all()
assert submission_df['annotation'].apply(lambda x: x=="authentic" or (x.startswith("[") and x.endswith("]"))).all()

# -----------------------
# SAVE CSV
# -----------------------
submission_df.to_csv('submission.csv', index=False)
print("✅ Submission file created successfully!")
print(submission_df.head())
print("Number of rows:", len(submission_df))

# Optionnel : statistiques
authentic_count = (submission_df['annotation'] == 'authentic').sum()
forged_count = len(submission_df) - authentic_count
print(f"Authentic: {authentic_count}, Forged: {forged_count}")


# Conclusion

<div style="font-family: -apple-system, BlinkMacSystemFont, 'Segoe UI', Roboto, 'Helvetica Neue', Arial; line-height:1.5; color:#0b1220; max-width:800px;">
  <h2 style="margin-bottom:6px;">🎯 The Challenge</h2>
  <p style="margin-top:0;">
    Scientific image forgery (copy-move) threatens research integrity by misleading scientists and wasting resources.
    The goal: build an automated model to detect and segment forged regions in biomedical images.
  </p>

  <h3 style="margin-bottom:6px;">⚙️ Our Approach</h3>
  <p style="margin-top:0;">
    A <strong>U-Net</strong> model with a <strong>ResNet34</strong> backbone trained on <strong>5,128 images</strong>
    (2,751 forged / 2,377 authentic). Used <strong>Dice + BCE loss</strong> to handle imbalance and achieved strong Dice performance on validation.
  </p>

  <h3 style="margin-bottom:6px;">💡 Key Insights</h3>
  <ul style="margin-top:0  ; padding-left:18px;">
    <li>Augmentation (rotations, flips, contrast) improves generalization</li>
    <li>Threshold tuning enhances accuracy</li>
  </ul>

  <h3 style="margin-bottom:6px;">🚀 Future Work</h3>
  <ul style="margin-top:0; padding-left:18px;">
    <li>Test advanced backbones (EfficientNet, Swin Transformer)</li>
    <li>Add attention and multi-scale prediction</li>
  </ul>

  <h3 style="margin-bottom:6px;">🌍 Impact</h3>
  <p style="margin-top:0;">
    Helps journals and researchers detect fraud early and restore trust in scientific imagery.
  </p>
</div>
